In [1]:
import findspark
import os
os.environ['SPARK_HOME'] = "C:\spark-3.0.3-bin-hadoop2.7"
findspark.init()

from pyspark.sql import SparkSession
import pyspark.sql.functions as F


In [2]:
spark = SparkSession.builder.appName("Stage").getOrCreate()

In [3]:
#spark

In [4]:
df = spark.read.format("csv").load('C:/Users/Ben Romdhane/Downloads/yellow_tripdata_2020-01.csv', header=True, InferSchema=True)

In [5]:
df.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge']

In [6]:
#df.head(5)

In [7]:
#df.describe()

In [8]:
data=df.dropDuplicates()

In [9]:
#df.count()

In [10]:
#data.count()

In [11]:
cols = ("RatecodeID","store_and_fwd_flag","payment_type","extra","mta_tax",
        "tolls_amount","improvement_surcharge","congestion_surcharge","fare_amount","tip_amount")

In [12]:
data=data.drop(*cols)

In [13]:
#data.show()

In [14]:
from pyspark.sql.functions import isnan, when, count, col
#data.select([count(when(isnan(c), c)).alias(c) for c in data.columns]).show()
#count of missing values

In [15]:
#data.select([count(when(col(c).isNull(), c)).alias(c) for c in data.columns]).show()
#count of null values

In [16]:
#data.filter(
#  data.passenger_count.between(0,0)).show()

In [17]:
newdata=data.dropna(how="any", subset=["VendorID","passenger_count"])

#newdata.select([count(when(col(c).isNull(), c)).alias(c) for c in newdata.columns]).show()


In [18]:
total = newdata.select('total_amount').distinct().collect()

In [19]:
total

[Row(total_amount=49.8),
 Row(total_amount=41.42),
 Row(total_amount=9.13),
 Row(total_amount=13.4),
 Row(total_amount=58.92),
 Row(total_amount=15.5),
 Row(total_amount=17.52),
 Row(total_amount=-12.3),
 Row(total_amount=10.65),
 Row(total_amount=14.9),
 Row(total_amount=128.8),
 Row(total_amount=46.15),
 Row(total_amount=69.8),
 Row(total_amount=43.03),
 Row(total_amount=61.78),
 Row(total_amount=56.8),
 Row(total_amount=12.32),
 Row(total_amount=53.82),
 Row(total_amount=70.07),
 Row(total_amount=114.36),
 Row(total_amount=19.98),
 Row(total_amount=26.7),
 Row(total_amount=40.53),
 Row(total_amount=98.3),
 Row(total_amount=26.72),
 Row(total_amount=40.11),
 Row(total_amount=86.44),
 Row(total_amount=76.46),
 Row(total_amount=45.43),
 Row(total_amount=74.5),
 Row(total_amount=65.78),
 Row(total_amount=17.56),
 Row(total_amount=85.86),
 Row(total_amount=300.96),
 Row(total_amount=213.3),
 Row(total_amount=37.81),
 Row(total_amount=38.61),
 Row(total_amount=17.95),
 Row(total_amount=50

In [20]:
#for t in total[:]:
 #   _df = newdata.where(f"total_amount = '{t[0]}'")
 #   _df.coalesce(1).write.mode("append").option("header", "true").csv("C:/Users/Ben Romdhane/Downloads/stage")

In [21]:
#newdata.coalesce(1).write.mode("append").option("header", "true").csv("C:/Users/Ben Romdhane/Downloads/test")

In [22]:
#f= spark.read.format("csv").load('C:/Users/Ben Romdhane/Downloads/test/part-00000-deabcb4c-09db-4153-83f0-0d30d3108ad2-c000.csv', header=True, InferSchema=True)


In [23]:
#f_schema=f.schema

In [24]:
part= spark.read.format("csv").load('C:/Users/Ben Romdhane/Downloads/stage/part-00000-00f19c47-9cc0-44a4-a042-633c5266f7e8-c000.csv', header=True, InferSchema=True)


In [25]:
part_schema=part.schema

In [26]:
initDF = (spark
  .readStream
  .format("csv")
  .option("maxFilesPerTrigger", 2) 
  .option("path", "C:/Users/Ben Romdhane/Downloads/stage")
  .schema(part_schema)
  .load()
  )

In [27]:
initDF.isStreaming

True

In [28]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [43]:
#p=newdata.toPandas()
#plt.plot(p['trip_distance'],p['total_amount'])

#p=newdata.toPandas()
#plt.hist(p["total_amount"])


ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:54393)
Traceback (most recent call last):
  File "C:\spark-3.0.3-bin-hadoop2.7\python\lib\py4j-0.10.9-src.zip\py4j\java_gateway.py", line 1193, in send_command
    self.socket.sendall(command.encode("utf-8"))
ConnectionResetError: [WinError 10054] Une connexion existante a dû être fermée par l’hôte distant

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\spark-3.0.3-bin-hadoop2.7\python\lib\py4j-0.10.9-src.zip\py4j\java_gateway.py", line 1033, in send_command
    response = connection.send_command(command)
  File "C:\spark-3.0.3-bin-hadoop2.7\python\lib\py4j-0.10.9-src.zip\py4j\java_gateway.py", line 1196, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\spark-3.0.3

Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:54393)

In [40]:
DF=initDF.groupBy("total_amount").count()

query=(DF.writeStream
.format("console")
.outputMode("update") 
.start()
)